# Extract Gene Sets
This section will contain functionality to extract gene sets from symbolic linked disease sets. The goal of this is that once gene sets are extracted, monte-carlo's can be performed on a locally run DGIdb to generate histogram's of returned drugs. This will identified gene sets (hopefully) as stastically significant and relevant to diseases.

In [1]:
import pandas as pd
import re
import requests
import json
import random
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# Define disease sets
disease_sets = ['brainCancer','hereditaryCancer','metabolicDiseases','renalDiseases','cardiacDisorders','cnsCancer','skinDiseases','t-All','wilmsTumor','MDS','epilepsy','pediatricCancer']

# Done: all
# Still too long URI: pediatricDisease

In [4]:
# Open yaml files to get excel names  (missing epilepsy & mds & pediatricCancer & pediatricDisease)

excel_file_list = list()
for i in disease_sets:
    with open('../symlinks/panels/'+ i + '.yaml') as f:
        individual_gene_panels = f.read().splitlines()
    excel_file_list.append(individual_gene_panels)



In [5]:
# Functions to get genes
def get_genes(df):
    
    gene_list = list()
    try:
        for i in df.geneSymbol:
            gene_list.append(i)
    except:
        for i in df.Gene:
            gene_list.append(i)

    return(gene_list)

In [6]:
# Open files and retrieve metrics

list_counter = 0

genes_df_list = list()
disease_df_list = list()
source_file_list = list()
genes_length_list = list()


while list_counter < len(excel_file_list):
    for i in excel_file_list[list_counter]:
        try:
            try:
                f = pd.read_excel('../../diseases/' + disease_sets[list_counter] + '/' + str(i),sheet_name='genes',index_col=0)
                genes = get_genes(f)

                genes_df_list.append(genes)
                genes_length_list.append(str(len(genes)))
                disease_df_list.append(disease_sets[list_counter])
                source_file_list.append(i)

            except:
                f = pd.read_excel('../../diseases/' + disease_sets[list_counter] + '/' + str(i),sheet_name='data',index_col=0)
                genes = get_genes(f)

                genes_df_list.append(genes)
                genes_length_list.append(str(len(genes)))
                disease_df_list.append(disease_sets[list_counter])
                source_file_list.append(i)

            print('Opened file: ' + i + ' for disease ' + disease_sets[list_counter])
        except:
            print ('../../diseases/' + disease_sets[list_counter] + '/' + str(i))

            genes_df_list.append('None')     
            genes_length_list.append('Nan')
            disease_df_list.append(disease_sets[list_counter])
            source_file_list.append(i)

    list_counter += 1

Opened file: Ambry Genetics BrainTumorNext ®.xlsx for disease brainCancer
Opened file: Baylor Miraca Genetics Laboratories Hereditary Brain CNS PNS Cancer Panel.xlsx for disease brainCancer
../../diseases/brainCancer/Children's Hospital of Philadelphia - Division of Genomic Diagnostics Hereditary Brain Tumor Panel.xlsx
Opened file: EGL Genetics Brain, CNS, and PNS Cancer Panel: Sequencing and CNV Analysis.xlsx for disease brainCancer
Opened file: EGL Genetics Brain, CNS, and PNS Cancer: Deletion Duplication Panel.xlsx for disease brainCancer
Opened file: Fulgent Genetics Nervous System   Brain Cancer Comprehensive Panel  Sequencing Only).xlsx for disease brainCancer
Opened file: BioReference Laboratories OnkoSight Glioma Panel.xlsx for disease brainCancer
Opened file: GeneDx Brain Tumor Panel.xlsx for disease brainCancer
Opened file: Integrated Genetics VistaSeq Brain   CNS   PNS Cancer Panel.xlsx for disease brainCancer
Opened file: Invitae Invitae Nervous System Brain Cancer Panel - 

In [7]:
genes_df = pd.DataFrame()
genes_df = genes_df.assign(disease=disease_df_list,source=source_file_list,genes=genes_df_list,number=genes_length_list)
genes_df

,disease,source,genes,number
0,brainCancer,Ambry Genetics BrainTumorNext ®.xlsx,"[AIP, ALK, APC, CDKN1B, CDKN2A, DICER1, EPCAM,...",29
1,brainCancer,Baylor Miraca Genetics Laboratories Hereditary...,"[ALK, APC, ATM, MEN1, MLH1, MRE11, MSH2, MSH6,...",17
2,brainCancer,Children's Hospital of Philadelphia - Division...,None,Nan
3,brainCancer,"EGL Genetics Brain, CNS, and PNS Cancer Panel:...","[ALK, APC, ATM, MEN1, MLH1, MSH2, MSH6, NBN, N...",15
4,brainCancer,"EGL Genetics Brain, CNS, and PNS Cancer: Delet...","[ALK, APC, ATM, MEN1, MLH1, MSH2, MSH6, NBN, N...",15
...,...,...,...,...
136,pediatricCancer,Invitae Invitae Pediatric Nervous System Brain...,[],0
137,pediatricCancer,Invitae Invitae Pediatric Nervous System Brain...,"[AIP, ALK, APC, DICER1, EPCAM, HRAS, LZTR1, ME...",26
138,pediatricCancer,Invitae Invitae Pediatric Solid Tumors Panel.xlsx,"[AIP, ALK, APC, AXIN2, BAP1, BLM, BMPR1A, CDC7...",53
139,pediatricCancer,PerkinElmer Genomics Pediatric Tumor Panel.xlsx,"[ALK, APC, CDC73, DICER1, EPCAM, MEN1, MLH1, M...",27


# Define Monte Carlo functionality
Gene sets have been extracted with appropriate meta data attached in one big dataframe. Build functionality to perform monte carlo simulations off local DGIdb instance using the length of gene list for sample size. Enable functionality for both source level and disease level? I think disease level is the focus

In [8]:
# Get gene set as defined by diseases
def get_gene_set(disease,df):

    df = df[df['disease']==disease]

    all_genes = list()

    for i in df.genes:
        if i == 'None':
            pass
        else:
            for j in i:
                all_genes.append(j)
    full_gene_set = list(set(all_genes))
    return(full_gene_set)

# Get drug JSON from locally run DGIdb  
def get_json(input_genes):

    input_genes = ','.join(input_genes)

    r = requests.get('http://localhost:3000/api/v2/interactions.json?genes=' + input_genes + '&fda_approved_drug=true')
    data = r.json()

    return(data)

# Get number of drugs from JSON (more data points can be added here)
def get_monte_carlo_data_points(data):

    number_of_drugs = 0

    for i in data['matchedTerms']:
        data_point = len(i['interactions'])
        number_of_drugs = number_of_drugs + data_point

    return(number_of_drugs)


In [8]:
lol = get_gene_set('hereditaryCancer',genes_df)
len(lol)

641

In [10]:
data = get_json(lol)

KeyboardInterrupt: 

In [12]:
result = get_monte_carlo_data_points(data)
result

2316

6

# Random sampling functionality
Random sampling functionality for simulations. Use previously made random gene sets.

In [9]:
def get_random_sample(length):

    with open('allGenes.yaml') as f:
        all_genes = f.read().splitlines()    

    sample = random.sample(all_genes, length)

    return(sample)

# For random sampling purposes, save json for later retrieval
def save_json(data,file_name):

    with open('json/' + file_name + '.json', 'w') as outfile:
        json.dump(data, outfile)

    pass

In [13]:
random_genes = get_random_sample(len(lol))
random_genes

['TCHH',
 'TSHRL3',
 'RNU2-1',
 'TRPM1',
 'NPTX2',
 'LINC01256',
 'NUP37',
 'INTS8',
 'ATP5J',
 'SP2-AS1',
 'RBP4',
 'CLYBL-AS1',
 'OR1L3',
 'RHCG',
 'DHRSX',
 'ZYG11A',
 'ITGB5',
 'TNNI1',
 'TMEM249',
 'HEATR9',
 'SYCP2',
 'LINC00244',
 'LINC00440',
 'ANKRD65',
 'LINC01123',
 'MET',
 'TRT-AGT2-1',
 'POSTN',
 'IGHV1-8',
 'ZNHIT3',
 'CYP4F22',
 'PRR20E',
 'SLC17A7',
 'TUBB4B',
 'DFNB68',
 'SNORD115-47',
 'SPG37',
 'ZBTB43',
 'C5orf45',
 'TCF15',
 'C11orf1',
 'CDK5RAP1',
 'AKIRIN1',
 'HEXB',
 'EPHA5',
 'DSCAM',
 'ZEB2-AS1',
 'KRTAP5-AS1',
 'MT3',
 'G6PC3',
 'TRBV10-2',
 'ZBP1',
 'CYP46A1',
 'CEP57',
 'FUT8',
 'PRSS51',
 'TUBB3',
 'MBL2',
 'IGHV4-30-4',
 'RPS24',
 'ERVK-21',
 'SLX1A',
 'RAB32',
 'LINC00462',
 'PIRC39',
 'NEURL1B',
 'MRPS26',
 'APOC2',
 'LINC01291',
 'GUSB',
 'PRPF3',
 'GRIN1',
 'VRK1',
 'TAPBPL',
 'PSMD3',
 'FAM171B']

In [96]:
random_data = get_json(random_genes)


In [95]:
random_result = get_monte_carlo_data_points(random_data)
random_result

93

# Putting it all together
Do an actual monte carlo simulation with 500 samples (499 + 1)

In [10]:

def run_monte_carlo_simulation(genes_df,disease):

    # Putting it all together 
    df = pd.DataFrame()
    sample_source_list = list()
    number_of_drugs_list = list()
    genes_input_list = list()

    # disease_sets has full list, but random samples length will change depending on gene set length

    # Primary samples
    primary_sample = get_gene_set(disease,genes_df)
    data = get_json(primary_sample)
    result = get_monte_carlo_data_points(data)

    sample_source_list.append(disease)
    number_of_drugs_list.append(str(result))
    genes_input_list.append(primary_sample)

    # Loop
    loop_counter = 1

    while loop_counter < 500:
        random_sample = get_random_sample(len(primary_sample))
        random_data = get_json(random_sample)
        random_result = get_monte_carlo_data_points(random_data)

        sample_source_list.append('random_sample')
        number_of_drugs_list.append(int(random_result))
        genes_input_list.append(random_sample)

        loop_counter += 1
        if loop_counter == 1 | 50 | 100 | 200 | 300 | 400 | 500:
            print(str('On iteration ' + loop_counter + 'for disease state: ' + disease))

    df = df.assign(sample_source=sample_source_list, number_of_drugs=number_of_drugs_list, genes_input=genes_input_list)

    # Write DataFrames to excel files by query input
    file_name = disease
    writer = pd.ExcelWriter('simulation_results/' + file_name + '.xlsx')
    df.to_excel(writer,sheet_name='results')
    print('Saving simulation result for ' + disease)
    writer.save()


    return(df)

In [13]:
for i in disease_sets:
    print(i)

hereditaryCancer
metabolicDiseases
renalDiseases
brainCancer
cardiacDisorders
cnsCancer
skinDiseases
t-All
wilmsTumor
MDS
epilepsy
pediatricCancer
pediatricDisease


In [51]:
for i in disease_sets:
    run_monte_carlo_simulation(genes_df,i)

Saving simulation result for hereditaryCancer
Saving simulation result for metabolicDiseases
Saving simulation result for renalDiseases


KeyboardInterrupt: 

# Plot resulting histogram
Simulation is ran and data is saved. Plot the results!

In [106]:
# quick transform to int on accident
temp = monte_carlo_df
index_counter = 0
for i in monte_carlo_df['number_of_drugs']:
    monte_carlo_df['number_of_drugs'][index_counter] = int(i)
    

In [11]:
# Get all P values

pvalues = pd.DataFrame()
pvalues_list = list()
disease_list = list()

for i in disease_sets:
    incidence_counter = 0
    pval_df = pd.read_excel('simulation_results/' + i + '.xlsx',sheet_name='results',index_col=0)
    test_value = pval_df['number_of_drugs'][0]
    incidence_counter += 1

    for j in pval_df['number_of_drugs']:
        if j >= test_value:
            incidence_counter += 1
        else:
            pass

    p_value = incidence_counter / len(pval_df['number_of_drugs'])
    pvalues_list.append(p_value)
    disease_list.append(i)

pvalues = pvalues.assign(pvalues = pvalues_list, diseases = disease_list)

pvalues

,pvalues,diseases
0,0.004,brainCancer
1,0.004,hereditaryCancer
2,0.004,metabolicDiseases
3,0.010,renalDiseases
4,0.022,cardiacDisorders
5,0.006,cnsCancer
6,0.004,skinDiseases
7,0.008,t-All
8,0.014,wilmsTumor
9,0.004,MDS


In [13]:
# Read data from file
filename = 'renalDiseases'
monte_carlo_df = pd.read_excel('simulation_results/' + filename + '.xlsx',sheet_name='results',index_col=0)
monte_carlo_df

,sample_source,number_of_drugs,genes_input
0,renalDiseases,2210,"['AHI1', 'CTU2', 'OCRL', 'AMN', 'FN1', 'ANKS6'..."
1,random_sample,742,"['CTIF', 'ST13', 'RAPGEF5', 'CENPE', 'STARD3',..."
2,random_sample,902,"['SRCAP', 'SMIM8', 'G6PC3', 'MEA1', 'PDK2', 'P..."
3,random_sample,666,"['CHCHD2', 'EIF4H', 'GYLTL1B', 'COPRS', 'VARS'..."
4,random_sample,1588,"['SUZ12', 'CTXN3', 'MGME1', 'RPP40', 'RPS15', ..."
...,...,...,...
495,random_sample,468,"['NPIPA8', 'CLEC1A', 'GLTPD2', 'WDR37', 'TRAV8..."
496,random_sample,767,"['NREP', 'TNIP2', 'MYOM2', 'LINC01193', 'FEZF2..."
497,random_sample,1642,"['PSMA2', 'LRRC3DN', 'STOX2', 'PIRC105', 'HMCN..."
498,random_sample,587,"['KRTAP5-3', 'DFNB60', 'FXR1', 'TMEM45B', 'ALD..."


In [18]:
# # Make figure
# fig = px.histogram(monte_carlo_df, x='number_of_drugs', color='sample_source', marginal='box')
# fig.show()
# pio.write_image(fig, 'test.pdf', width=3*300, height=3*250, scale=3)

# Figure loop
for i in disease_sets:
    monte_carlo_df = pd.read_excel('simulation_results/' + i + '.xlsx',sheet_name='results',index_col=0)

    fig = fig = px.histogram(monte_carlo_df, x='number_of_drugs', color='sample_source', marginal='box')
    pio.write_image(fig,'graphs/' + i + '.pdf', width=3*400, height=3*250, scale=3)


In [28]:
# Make figure
fig = px.histogram(monte_carlo_df, x='number_of_drugs', color='sample_source', marginal='box')
fig.show()

In [31]:
# Make figure
fig = px.histogram(monte_carlo_df, x='number_of_drugs', color='sample_source', marginal='box')
fig.show()